<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Enhanced_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import mip
except ImportError:
    import sys
    !{sys.executable} -m pip install mip
import pandas as pd
import numpy as np
import re
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 30.1 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


# Create Player dataset

In [2]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path)#, on_bad_lines='skip')
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

def extract_numbers(s):
    if pd.isna(s):
        return None
    numbers = re.findall(r'\d+', str(s))
    if numbers:
        return int(numbers[0])
    return None

num_of_players = len(df) #250

projections_df = df[['Player', 'Bye', 'POS', 'AVG']].head(num_of_players).copy()
projections_df['Bye'] = projections_df['Bye'].apply(extract_numbers)
projections_df['Bye'] = projections_df['Bye'].fillna(0).astype(int)

projections_df.info()
print(projections_df.value_counts('POS'))
print(projections_df.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  948 non-null    object 
 1   Bye     948 non-null    int64  
 2   POS     948 non-null    object 
 3   AVG     948 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 29.8+ KB
POS
WR     335
RB     224
TE     166
QB     127
K       64
DST     32
Name: count, dtype: int64
                 Player  Bye POS   AVG
0   Christian McCaffrey    9  RB   1.0
1           CeeDee Lamb    7  WR   2.6
2           Tyreek Hill    6  WR   3.2
3        Bijan Robinson   12  RB   5.0
4           Breece Hall   12  RB   5.4
5     Amon-Ra St. Brown    5  WR   6.2
6         Ja'Marr Chase   12  WR   6.6
7      Justin Jefferson    6  WR   7.0
8        Saquon Barkley    5  RB   9.2
9            A.J. Brown    5  WR  10.2
10      Jonathan Taylor   14  RB  10.4
11       Garrett Wilson   12  WR  12.4
12      

# Create custom projections
Maybe delete later when real projections available

In [3]:
# Anzahl der Wochen
number_of_weeks = 17

# Wochen-Spaltennamen
weekly_columns = [f"Week_{i+1}" for i in range(number_of_weeks)]

# Skalierungsfunktion
def projection_base(avg, pos, max_val=22, min_val=7, k=75, c=1.5):
    proj = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
    if pos == "QB":
        proj += 8  # QB-Bonus
    elif pos == "K":
        proj -= 2  # K-Penalty
    elif pos == "DST":
        proj -= 3  # DST-Penalty
    return proj

final_projections = []
for _, row in projections_df.iterrows():
    base_score = projection_base(row['AVG'], row['POS'])

    # Erstellen der weekly projections
    weekly_proj = []
    for week in range(number_of_weeks):
        # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
        if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
            weekly_proj.append(0.0)  # Projektion auf 0 setzen
        else:
            weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

    final_projections.append(weekly_proj)

# Projektionen in DataFrame einfügen
f = projections_df[["Player", "Bye", "POS", "AVG"]].copy()
for i, col in enumerate(weekly_columns):
    f[col] = [proj[i] for proj in final_projections]
f["TTL"] = f[weekly_columns].sum(axis=1)
f['dropoff'] = (f.sort_values(['POS','TTL'], ascending=[True, False]).groupby('POS')['TTL'].diff(-1).fillna(0.0))


# Stichprobe
probe = 4
print(f'Länge von f: {len(f)}')
print(f'head({probe}) of f:')
print()
# print(f.head(probe))
f.head(probe)

Länge von f: 948
head(4) of f:



,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,TTL,dropoff
0,Christian McCaffrey,9,RB,1.0,26.253013,19.866398,22.103500,22.445008,19.002930,24.672583,...,22.481375,22.157667,26.858950,23.640470,22.562003,24.887011,24.476146,22.468545,368.831207,22.119906
1,CeeDee Lamb,7,WR,2.6,26.738351,22.867702,17.390309,21.661888,25.365644,25.394579,...,21.833608,23.669178,24.051487,23.532357,27.535988,24.971728,21.411276,22.010434,373.053646,14.253919
2,Tyreek Hill,6,WR,3.2,20.243641,24.767232,22.341140,19.254944,19.625881,0.000000,...,23.646990,22.703963,23.935349,21.351426,20.159415,22.805430,20.728904,21.270330,350.334827,3.795671
3,Bijan Robinson,12,RB,5.0,24.866547,21.563859,18.213851,19.933337,17.407263,20.570153,...,21.124178,21.583874,0.000000,16.524703,21.135669,19.921694,22.359530,19.058021,328.669140,1.714663


In [4]:
# VOR berechnen


In [5]:
#filter for Player == Rico Dowdle
f[f['Player'] == 'Rico Dowdle']

,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,TTL,dropoff
148,Rico Dowdle,11,RB,150.8,10.651377,10.323023,10.333401,10.197524,11.551469,11.993792,...,10.703526,0.0,10.539686,13.122069,10.946352,10.689891,10.374568,12.911896,179.695596,0.280764


In [6]:
# ==========================
# 1. PARAMETERS
# ==========================

players = projections_df['Player'].tolist()
positions = {"QB", "RB", "WR", "TE", "K", "DST"}
weeks = list(range(1, 18))
pos = dict(zip(projections_df['Player'], projections_df['POS']))
pos_limit = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DST": 1}
week_cols = [col for col in f.columns if col.startswith("Week_")]

# Build f_dict: weekly projections and dropoff for each player
f_dict = {
    row['Player']: {
        **{int(week.replace("Week_", "")): row[week] for week in week_cols},
        'dropoff': row['dropoff']
    }
    for _, row in f.iterrows()
}

beta = {t: 140.0 for t in weeks}
gamma = {"QB": 2, "RB": 3, "WR": 3, "TE": 2, "K": 1, "DST": 1}
alpha, lambda_0, lambda_1, lambda_2, lambda_3 = 1.0, 100, 50, 75, 0
df_sorted = projections_df.sort_values("AVG").reset_index(drop=True)
df_sorted["Rank"] = df_sorted.index + 1
topk_pct = 0.0025
min_pos_req = pos_limit.copy()  # Minimum roster requirements per position

# ==========================
# 2. INITIALIZATION (TEAMS, DRAFT ORDER, OPPONENT PICK)
# ==========================

num_teams = 12
teams = [f"Team {i+1}" for i in range(num_teams)]
DM_team = "Team 3"
num_rounds = 15

# Snake draft order
draft_order = []
for rnd in range(num_rounds):
    order = teams if rnd % 2 == 0 else teams[::-1]
    draft_order += order

def opponent_pick(roster, available, Rk, min_pos_req, topk_pct=0.01):
    # 1) Sort remaining players by rank
    rem = sorted(available, key=lambda p: Rk[p])
    topk = max(1, int(len(rem) * topk_pct))

    # 2) Calculate deficits per position (min requirement minus current roster)
    deficits = {
        j: min_pos_req[j] - sum(1 for p in roster if pos[p] == j)
        for j in min_pos_req
    }
    needed = [j for j, d in deficits.items() if d > 0]

    # 3) If deficits exist, pick from candidates in those positions
    if needed:
        candidates = [p for p in rem if pos[p] in needed]
        pool = candidates[:topk] if len(candidates) >= topk else candidates
        if pool:
            return random.choice(pool)

    # 4) Fallback: pick randomly from top-k overall
    return random.choice(rem[:topk])

# ==========================
# 3. DRAFT INITIALIZATION
# ==========================

rosters = {tm: [] for tm in teams}   # Dict: team -> list of drafted players
available = set(players)             # Set of available players
draft_log = []                       # List to store draft results

# ==========================
# 4. MAIN DRAFT LOOP
# ==========================

for pick_idx, team in enumerate(draft_order, start=1):

    # ---- 4.1: Update remaining player ranking ----
    rem = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player == p, "AVG"].item())
    Rk = {p: i+1 for i, p in enumerate(rem)}
    player_vars = set(rem) | set(rosters[team])
    picks_remaining = num_rounds - len(rosters[team])

    # ---- 4.2: DM-Team (your team) picks via MIP ----
    if team == DM_team:
        m = Model(sense=maximize, solver_name="CBC")

        # --- Decision variables ---
        # y[i]: 1 if player i is drafted by DM, 0 otherwise
        y = {i: m.add_var(var_type=BINARY, name=f"y_{i}") for i in player_vars}
        # x[i, t]: share of player i's points used in week t (continuous)
        x = {(i, t): m.add_var(var_type=CONTINUOUS, name=f"x_{i}_{t}") for i in rem for t in weeks}
        # z[t]: 1 if DM wins in week t, 0 otherwise
        z = {t: m.add_var(var_type=BINARY, name=f"z_{t}") for t in weeks}

        # --- Objective function ---
        m.objective = (
            lambda_0 * xsum(f_dict[i][t] * x[i, t] for i in rem for t in weeks) # Total points
            + lambda_1 * xsum(z[t] for t in weeks[:15])                         # Early win indicator
            + lambda_2 * xsum(z[t] for t in weeks[15:])                         # Late win indicator
            + lambda_3 * xsum(f_dict[i]['dropoff'] * y[i] for i in rem)         # Dropoff bonus
        )

        # --- Constraints ---
        # Fix previous picks (already drafted players must remain picked)
        for p in rosters[team]:
            m += y[p] == 1

        # Enforce that exactly the remaining picks are made
        m += xsum(y[i] for i in rem) == picks_remaining

        # Enforce minimum requirements for each position (relative to already drafted players)
        for pos_name, req in min_pos_req.items():
            already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
            need = max(0, req - already_satisfied)
            m += xsum(y[i] for i in rem if pos[i] == pos_name) >= need

        # Weekly lineup constraints and position limits
        for j in positions:
        #     m += xsum(y[i] for i in rem if pos[i] == j) >= gamma[j]  # Minimum number per position
            for t in weeks:
                m += xsum(x[i, t] for i in rem if pos[i] == j) <= pos_limit[j]  # Weekly lineup limit

        # Only drafted players can be in the weekly lineup
        for i in rem:
            for t in weeks:
                m += x[i, t] <= y[i]

        # Win indicator constraints
        for t in weeks:
            m += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in rem) / beta[t]

        # Robust draft constraint (to simulate uncertainty in opponent picks)
        n_k = pick_idx
        for future_pick in range(
            pick_idx + 1,
            pick_idx + picks_remaining * len(teams),
            len(teams)
        ):
            top_cut = int(alpha * (future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i, r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ((future_pick - n_k) // len(teams))

        # --- Solve the MIP model ---
        m.optimize()
        # Check for infeasibility
        if m.num_solutions == 0:
            raise RuntimeError(f"No feasible solution at pick {pick_idx}. Check constraints and player pool.")

        # Extract chosen player for this pick
        chosen = [i for i in rem if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f"No feasible pick at {pick_idx}")
        pick = min(chosen, key=lambda i: Rk[i])

    # ---- 4.3: Opponent pick (simple heuristic) ----
    else:
        current_round = (pick_idx - 1) // len(teams) + 1
        dynamic_topk_pct = min(current_round * 2 * topk_pct, 1.0)
        pick = opponent_pick(
            roster=rosters[team],
            available=available,
            Rk=Rk,
            min_pos_req=pos_limit,
            topk_pct=dynamic_topk_pct
        )

    # ---- 4.4: Update rosters and draft log ----
    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        "Pick": pick_idx,
        "Team": team,
        "Player": pick,
        "Round": (pick_idx - 1) // len(teams) + 1,
        "POS": pos[pick]
    })

# ==========================
# 5. CREATE DRAFT DATAFRAME
# ==========================

df_draft = pd.DataFrame(draft_log)
print(df_draft.head(1 + num_teams * 2))


    Pick     Team               Player  Round POS
0      1   Team 1          CeeDee Lamb      1  WR
1      2   Team 2       Bijan Robinson      1  RB
2      3   Team 3         Alvin Kamara      1  RB
3      4   Team 4  Christian McCaffrey      1  RB
4      5   Team 5          Tyreek Hill      1  WR
5      6   Team 6     Justin Jefferson      1  WR
6      7   Team 7        Ja'Marr Chase      1  WR
7      8   Team 8           A.J. Brown      1  WR
8      9   Team 9       Saquon Barkley      1  RB
9     10  Team 10          Breece Hall      1  RB
10    11  Team 11       Garrett Wilson      1  WR
11    12  Team 12         Jahmyr Gibbs      1  RB
12    13  Team 12    Amon-Ra St. Brown      2  WR
13    14  Team 11        Isiah Pacheco      2  RB
14    15  Team 10           Puka Nacua      2  WR
15    16   Team 9  Marvin Harrison Jr.      2  WR
16    17   Team 8         Travis Kelce      2  TE
17    18   Team 7       Kyren Williams      2  RB
18    19   Team 6      Jonathan Taylor      2  RB


In [7]:
merged = pd.merge(df_draft, f[['Player', 'TTL']], on='Player', how='left')

# Replace missing TTLs (e.g., für Spieler ohne Projection) mit 0
merged['TTL'] = merged['TTL'].fillna(0)

# Group by Team and sum TTL to get total projection per team
team_ttl_proj = merged.groupby('Team')['TTL'].sum().reset_index()
team_ttl_proj = team_ttl_proj.rename(columns={'TTL': 'TTL_proj'})
team_ttl_proj

,Team,TTL_proj
0,Team 1,3488.437142
1,Team 10,3382.267353
2,Team 11,3413.845420
3,Team 12,3273.709005
4,Team 2,3323.775795
5,Team 3,2865.288497
6,Team 4,3416.174364
7,Team 5,3653.657395
8,Team 6,3945.972419
9,Team 7,3554.975311


In [8]:
# # --- 1. Parameter ---
# players        = projections_df['Player'].tolist()
# positions      = {"QB", "RB", "WR", "TE", "K", "DST"}
# weeks          = list(range(1,18))
# pos            = dict(zip(projections_df['Player'], projections_df['POS']))
# pos_limit      = {"QB":1,"RB":2,"WR":2,"TE":1,"K":1,"DST":1}
# week_cols = [col for col in f.columns if col.startswith("Week_")]
# f_dict = {
#     row['Player']: {**{int(week.replace("Week_", "")): row[week] for week in week_cols}, 'dropoff': row['dropoff']}
#     for _, row in f.iterrows()
# }
# beta           = {t:140.0 for t in weeks}
# gamma          = {"QB":2,"RB":3,"WR":3,"TE":2,"K":1,"DST":1}
# alpha, lambda_0, lambda_1, lambda_2, lambda_3 = 1.0, 1, 100, 150, 25
# df_sorted = projections_df.sort_values("AVG").reset_index(drop=True)
# df_sorted["Rank"] = df_sorted.index + 1
# topk_pct = 0.005
# min_pos_req = pos_limit


# # --- 2. Teams, DM-Team und Snake-Draft ---
# num_teams      = 12
# teams          = [f"Team {i+1}" for i in range(num_teams)]
# DM_team        = "Team 3"
# num_rounds     = 15
# draft_order    = []
# for rnd in range(num_rounds):
#     order = teams if rnd % 2 == 0 else teams[::-1]
#     draft_order += order

# def opponent_pick(roster, available, Rk, min_pos_req, topk_pct=0.01):
#     # 1) verbleibende Spieler neu sortieren
#     rem  = sorted(available, key=lambda p: Rk[p])
#     topk = max(1, int(len(rem) * topk_pct))

#     # 2) Defizite je Position (Mindestsoll minus aktueller Bestand)
#     deficits = {
#         j: min_pos_req[j] - sum(1 for p in roster if pos[p] == j)
#         for j in min_pos_req
#     }
#     needed = [j for j, d in deficits.items() if d > 0]

#     # 3) solange Defizite bestehen, aus allen rem dieser Position picken
#     if needed:
#         # Kandidaten aller benötigten Positionen
#         candidates = [p for p in rem if pos[p] in needed]
#         # begrenze auf Top-k, falls mehr Kandidaten vorhanden
#         pool = candidates[:topk] if len(candidates) >= topk else candidates
#         if pool:
#             return random.choice(pool)

#     # 4) Fallback: zufällig aus Top-k aller Positionen
#     return random.choice(rem[:topk])

# # --- 3. Initialisierung ---
# rosters        = {tm: [] for tm in teams}
# available      = set(players)
# draft_log      = []

# # --- 4. Optimierung über alle Picks ---
# for pick_idx, team in enumerate(draft_order, start=1):
#     # 4.1 Ranking der verbleibenden Spieler aktualisieren
#     rem = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player==p,"AVG"].item())
#     Rk  = {p: i+1 for i,p in enumerate(rem)}
#     player_vars = set(rem) | set(rosters[team])
#     picks_remaining = num_rounds - len(rosters[team])


#     # 4.2 DM-Pick via MIP
#     if team == DM_team:
#         m = Model(sense=maximize, solver_name="CBC")

#         # Entscheidungsvariablen
#         y = {i: m.add_var(var_type=BINARY, name=f"y_{i}")
#               for i in player_vars}
#         x = {(i,t): m.add_var(var_type=CONTINUOUS, name=f"x_{i}_{t}")
#              for i in rem for t in weeks}
#         z = {t: m.add_var(var_type=BINARY, name=f"z_{t}") for t in weeks}

#         # Objective Function
#         m.objective = (
#             lambda_0 * xsum(f_dict[i][t]*x[i,t] for i in rem for t in weeks)
#           + lambda_1 * xsum(z[t] for t in weeks[:15])
#           + lambda_2 * xsum(z[t] for t in weeks[15:])
#           + lambda_3 * xsum(f_dict[i]['dropoff'] * y[i] for i in rem)
#         )

#         # Constraints
#         # Fixiere vergangene Picks
#         for p in rosters[team]:
#             m += y[p] == 1

#         # Exakte Anzahl verbleibender Picks
#         m += xsum(y[i] for i in rem) == picks_remaining

#         # Mindestanforderungen relativ zu schon gezogenen Spielern
#         for pos_name, req in min_pos_req.items():
#             already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
#             need = max(0, req - already_satisfied)
#             m += xsum(y[i] for i in rem if pos[i] == pos_name) >= need

#         # Position‐ und Roster‐Constraints
#         for j in positions:
#             m += xsum(y[i] for i in rem if pos[i]==j) >= gamma[j]
#             for t in weeks:
#                 m += xsum(x[i,t] for i in rem if pos[i]==j) <= pos_limit[j]

#         # # Roster Anforderungen
#         # for pos_name, req in min_pos_req.items():
#         #     m.add_constr(
#         #         xsum(y[i] for i in player_vars if pos.get(i) == pos_name) >= req,
#         #         name=f"min_roster_{pos_name}")
#         # # maximale Picks pro Team
#         # m += xsum(y[i] for i in rem) <= num_rounds

#         # nur gedraftete Spieler in der Week‐Lineup
#         for i in rem:
#             for t in weeks:
#                 m += x[i,t] <= y[i]

#         # Win‐Indicator
#         for t in weeks:
#             m += z[t] <= xsum(f_dict[i][t]*x[i,t] for i in rem) / beta[t]

#         # Robuste Draft‐Constraint
#         n_k = pick_idx
#         for future_pick in range(pick_idx+1, pick_idx + (num_rounds - len(rosters[team]))*len(teams), len(teams)):
#             top_cut = int(alpha*(future_pick - n_k))
#             if top_cut > 0:
#                 top_players = [i for i,r in Rk.items() if r <= top_cut]
#                 m += xsum(y[i] for i in top_players) <= ( (future_pick-n_k) // len(teams) )

#         m.optimize()
#         # if m.num_solutions == 0:
#         #     raise RuntimeError(f"No feasible solution at pick {pick_idx}. Check constraints and player pool.")

#         # chosen = [i for i in rem if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
#         # if not chosen:
#         #     raise RuntimeError(f"No feasible pick at {pick_idx}")
#         # pick = min(chosen, key=lambda i: Rk[i])

#         # gewählten Spieler extrahieren
#         chosen = [i for i in rem if y[i].x >= 0.99 and i not in rosters[team]]
#         if not chosen:
#             raise RuntimeError(f"No feasible pick at {pick_idx}")
#         pick = min(chosen, key=lambda i: Rk[i])

#     # 4.3 Gegner-Pick: zufällig aus Top-5 verbleibend
#     else:
#         pick = opponent_pick(roster=rosters[team], available=available, Rk=Rk, min_pos_req=pos_limit, topk_pct=topk_pct)

#     # 4.4 Update
#     rosters[team].append(pick)
#     available.remove(pick)
#     draft_log.append({
#         "Pick": pick_idx, "Team": team, "Player": pick,
#         "Round": (pick_idx-1)//len(teams)+1, "POS": pos[pick]
#     })

# # --- 5. Ergebnis als DataFrame ---
# df_draft = pd.DataFrame(draft_log)
# print(df_draft.head(1+num_teams*2))

In [9]:
print(m.status)
#print(m.num_constrs, m.num_vars)
print(m)

OptimizationStatus.OPTIMAL


In [10]:
position_counts_df = df_draft.groupby(['Team', 'POS']).size().unstack(fill_value=0)
position_counts_df

POS,DST,K,QB,RB,TE,WR
Team,,,,,,
Team 1,1,1,1,4,3,5
Team 10,1,1,1,2,2,8
Team 11,2,1,2,3,2,5
Team 12,3,1,1,4,4,2
Team 2,2,2,1,4,1,5
Team 3,1,3,1,2,3,5
Team 4,2,2,1,2,2,6
Team 5,1,1,3,7,1,2
Team 6,2,1,4,3,1,4


In [11]:
result_dfs = {}

for team in df_draft['Team'].unique():
    team_df = df_draft[df_draft['Team'] == team].copy()
    team_df = team_df.sort_values(by='Pick')

    # Optional: Spalten anpassen, wenn nur bestimmte Infos gewünscht sind
    team_df['Pick Info'] = team_df.apply(lambda row: f"Round {row['Round']} Pick {row['Pick']}", axis=1)
    result_dfs[team] = team_df[['Player', 'Pick Info', 'POS']]  # oder andere gewünschte Spalten

for team, df in result_dfs.items():
    print(f"=== {team} ===")
    print(df)
    print()


=== Team 1 ===
                  Player          Pick Info  POS
0            CeeDee Lamb     Round 1 Pick 1   WR
23         Davante Adams    Round 2 Pick 24   WR
24             Joe Mixon    Round 3 Pick 25   RB
47        Raheem Mostert    Round 4 Pick 48   RB
48       Harrison Butker    Round 5 Pick 49    K
71          Trey McBride    Round 6 Pick 72   TE
72        Caleb Williams    Round 7 Pick 73   QB
95      Tennessee Titans    Round 8 Pick 96  DST
96           Tee Higgins    Round 9 Pick 97   WR
119        Xavier Worthy  Round 10 Pick 120   WR
120  Rhamondre Stevenson  Round 11 Pick 121   RB
143        Luke Musgrave  Round 12 Pick 144   TE
144     Christian Watson  Round 13 Pick 145   WR
167       Khalil Herbert  Round 14 Pick 168   RB
168        Tyler Conklin  Round 15 Pick 169   TE

=== Team 2 ===
                  Player          Pick Info  POS
1         Bijan Robinson     Round 1 Pick 2   RB
22         Derrick Henry    Round 2 Pick 23   RB
25           Jalen Hurts    Round 3 Pi